# Shopper Gaze Monitor

This shopper gaze monitor application is one of a series of reference implementations for Computer Vision (CV) using the Intel® Distribution of OpenVINO™ toolkit. It is designed for a retail shelf mounted camera system that counts the number of passers-by that look towards the display vs. the number of people that pass by the display without looking. It is intended to provide real-world marketing statistics for in-store shelf-space advertising.

# How it works

The application uses a video source, such as a camera, to grab frames, and then uses 2 different Deep Neural Networks (DNNs) to process the data. The first neural network detect faces. If the person's face is detected, it is counted as a "shopper".

A second neural network is then used to determine the head pose detection for each detected face. If the person's head is facing towards the camera, it is counted as a "looker".

The data can then optionally be sent to a MQTT machine to machine messaging server, as part of a retail data analytics system.

The DNN models can be downloaded using ./downloader.py present in model downloader folder that is part of the Intel® Distribution of OpenVINO™ toolkit.


The program creates three threads for concurrency:

 ->main thread that performs the video i/o
 ->worker thread that processes video frames using the deep neural networks
 ->worker thread that publishes any MQTT messages
 
 
 ## Step 0: Set Up
 
 ### 0.1: Import dependencies
    
Run the below cell to import Python dependencies needed for displaying the results in this notebook (tip: select the cell and use Ctrl+enter to run the cell)


In [1]:
from IPython.display import HTML
import matplotlib.pyplot as plt
import os
import time
import sys
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent.parent))
from demoTools.demoutils import *

# Building the code

Start by changing the current directory to wherever you have the application code and build it

In [ ]:
cd /home/u26213/Reference-samples/iot-devcloud/cpp/shopper_gaze_monitor

In [ ]:
rm -rf build && mkdir build

In [2]:
cd build

/home/u33139/DEVC_17_dec23/iot-devcloud/cpp/shopper_gaze_monitor/build


In [ ]:
!cmake ..

In [ ]:
!make

The above produces an executable monitor. This executable takes in a number of different command line arguments.

Run the following cell to see the list:

In [3]:
%%bash
./monitor -h

Starting main
After Commandline parser
Arg count:2
Use this script to using OpenVINO.
Usage: monitor [params] 

	-d, --device
		Device to run the inference (CPU, GPU, MYRIAD, FPGA or HDDL only).
	-f, --flag
		flag to run on sync or async mode.
	-h, --help (value:true)
		Print help message.
	-i, --input
		Path to input image or video file.
	-m, --model
		Path to .xml file of model containing face recognizer.
	-o, --output
		Path to output file.
	--pm, --posemodel
		Path to .xml file of face pose model.
	-r, --rate (value:1)
		number of seconds between data updates to MQTT server.



The version of the cpp file here is a slightly modified version of the shopper gaze monitor code built-in to the the Intel® Distribution of OpenVINO™ toolkit. In this version, the result is written into a output mp4 file specified by the -o flag.

## Step 1: using OpenVINO

## Downloading IR Model

The Intel® Distribution of OpenVINO™ toolkit comes with model downloader that can download pre-compiled Intermediate Representation (IR) models that are optimized for different end-point target devices. These models can be created from existing DNN models from popular frameworks (e.g. Caffe*, TF) using the Model Optimizer.

In [ ]:
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --print_all

Note the '!' is a special Jupyter Notebook command that allows you to run shell commands as if you are in a commannd line. So the above command will work straight out of the box on in a terminal (with '!' removed).

In this demo we will be using the 4 different types of models, with two model files each for FP16 and FP32. These models can be downloaded with the following command.

In [ ]:
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name face-detection-adas-0001 -o models
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name face-detection-adas-0001-fp16 -o models
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name head-pose-estimation-adas-0001 -o models
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name head-pose-estimation-adas-0001-fp16 -o models
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name age-gender-recognition-retail-0013 -o models
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name age-gender-recognition-retail-0013-fp16 -o models
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name emotions-recognition-retail-0003 -o models
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name emotions-recognition-retail-0003-fp16 -o models

The input arguments are as follows:

--name : name of the model you want to download. It should be one of the models listed in the previous cell.
-o : output directory. If this directory does not exist, it will be created.
There are more arguments to this script and you can get the full list using the -h option.

Running the inference
Now we are ready to run the inference workload. In this step, we will be submitting the workload as a job to the job queue.

## Step 2. Running the Inference

Currently, you are on what is called a "devnode". On this system, you are allocated just one core on a large Intel® Xeon® CPU. The purpose of this node is to develop code on the devnode and run minimal sections of Jupyter* Notebooks, but it is not meant to compute intensive jobs like deep learning inference. So we need to request additional resources from the cluster of Edge nodes to run the inference and this is done through the job queue.

To put an item on the job queue, we must first create a bash script that run the workload we want. Run the following cell to create bash script shopper_gaze.sh which will be our job script.

In [16]:
%%writefile shopper_gaze.sh
#PBS
INPUT_FILE=$1
OUTPUT_FILE=$2
FP_MODEL=$3
BACKEND=$4
#0: CPU target (by default), 1: OpenCL, 2: OpenCL fp16 (half-float precision), 3: VPU,5: HETERO:FPGA,CPU
TARGET=$5


if [ "$TARGET" == "5" ]; then
    # Environment variables and compilation for edge compute nodes with FPGAs
    source /opt/intel/init_openvino.sh
    aocl program acl0 /opt/intel/openvino/bitstreams/a10_vision_design_sg1_bitstreams/2019R3_PV_PL1_FP16_MobileNet_Clamp.aocx
fi

cd $PBS_O_WORKDIR
export LD_LIBRARY_PATH=${LD_LIBRARY_PATH}:/opt/intel/openvino/deployment_tools/inference_engine/lib/intel64/

 
if [ "$FP_MODEL" == "FP16" ]; then
  FPEXT='-fp16'
fi

    
./monitor \
-m="/home/u33139/DEVC_17_dec23/iot-devcloud/cpp/shopper_gaze_monitor/build/models/intel/face-detection-adas-0001/FP32/face-detection-adas-0001.xml" \
-pm="/home/u33139/DEVC_17_dec23/iot-devcloud/cpp/shopper_gaze_monitor/build/models/intel/head-pose-estimation-adas-0001/FP32/head-pose-estimation-adas-0001.xml" \
-i="/home/u33139/DEVC_17_dec23/iot-devcloud/cpp/shopper_gaze_monitor/resources/face-demographics-walking-and-pause.mp4" 


Overwriting shopper_gaze.sh


To put this script on the job queue, we use the command qsub. There are two important arguments we use with this command.

First, the -l flag. This flag is used to specify what type of resources to request from the cluster. For example this can be used to request an Intel® Xeon® CPU based system, or it can be used to request a system with an FPGA accelerator card in it. The syntax is -l nodes=1:<tag> where <tag> is the descriptor tag for the resource you want. For example, -l nodes=1:tank-870:e3-1268l-v5 will request an Intel® Xeon® system. To see the list of available tags, and the number of avilable systems, run the following cell.

In [9]:
!pbsnodes | grep compnode | sort | uniq -c

     35      properties = idc001skl,compnode,iei,tank-870,intel-core,i5-6500te,skylake,intel-hd-530,ram8gb,net1gbe
     15      properties = idc002mx8,compnode,iei,tank-870,intel-core,i5-6500te,skylake,intel-hd-530,ram8gb,net1gbe,hddl-r,iei-mustang-v100-mx8
     17      properties = idc003a10,compnode,iei,tank-870,intel-core,i5-6500te,skylake,intel-hd-530,ram8gb,net1gbe,hddl-f,iei-mustang-f100-a10
     23      properties = idc004nc2,compnode,iei,tank-870,intel-core,i5-6500te,skylake,intel-hd-530,ram8gb,net1gbe,ncs,intel-ncs2
      6      properties = idc006kbl,compnode,iei,tank-870,intel-core,i5-7500t,kaby-lake,intel-hd-630,ram8gb,net1gbe
     13      properties = idc007xv5,compnode,iei,tank-870,intel-xeon,e3-1268l-v5,skylake,intel-hd-p530,ram32gb,net1gbe
     15      properties = idc008u2g,compnode,up-squared,grove,intel-atom,e3950,apollo-lake,intel-hd-505,ram4gb,net1gbe,ncs,intel-ncs2
      1      properties = idc009jkl,compnode,jwip,intel-core,i5-7500,kaby-lake,intel-hd-630,ram8gb,n

Then there is the -F flag, which is used to pass in arguments to the job script. The face_detection_demo.sh takes in 2 arguments: 1) the path to the video to run inference on. 2) targeted device (CPU,GPU,MYRIAD). The job scheduler will use the contents of -F flag as the argument to the job script.

The following line will request an Intel Xeon system, and passes in "faces-recognition-walking.mp4 CPU" to the job script. Run the cell to submit this job.

In [10]:
os.environ["VIDEO"] = "../resources/face-demographics-walking-and-pause.mp4"

### Submitting to an edge compute node with an Intel® Core™ CPU

In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank 870-Q170</a> edge node with an <a 
    href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel 
    Core i5-6500TE</a>. The inference workload will run on the CPU.

In [17]:
print("Submitting job to Intel Core CPU...")
#Submit job to the queue
job_id_core = !qsub shopper_gaze.sh -l nodes=1:tank-870:i5-6500te -F "$VIDEO ../results/ FP32 CPU 0"
print(job_id_core[0])

#Progress indicators
if job_id_core:
    progressIndicator('../results/', 'i_progress_'+job_id_core[0]+'.txt', "Inference", 0, 100)

Submitting job to Intel Core CPU...
9700.v-qsvr-1.devcloud-edge


### Submitting to an edge compute node with Intel® Xeon® CPU
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank* 870-Q170</a> edge node with an <a 
    href="https://ark.intel.com/products/88178/Intel-Xeon-Processor-E3-1268L-v5-8M-Cache-2-40-GHz-">Intel® 
    Xeon® Processor E3-1268L v5</a>. The inference workload will run on the CPU.

In [ ]:
print("Submitting job to Intel Xeon CPU...")
#Submit job to the queue
job_id_xeon = !qsub shopper_gaze.sh -l nodes=1:tank-870:e3-1268l-v5 -F "$VIDEO ../results/ FP32 2 0"
print(job_id_xeon[0]) 
#Progress indicators
if job_id_xeon:
    progressIndicator('../results/', 'i_progress_'+job_id_xeon[0]+'.txt', "Inference", 0, 100)

### Submitting to an edge compute node with Intel® Core™ CPU and using the onboard Intel GPU

In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank 870-Q170</a> edge node with an <a href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel Core i5-6500TE</a>. The inference workload will run on the Intel® HD Graphics 530 card integrated with the CPU.

In [ ]:
print("Submitting job to Intel Core CPU with Intel GPU... FP32")
#Submit job to the queue
job_id_gpu = !qsub shopper_gaze.sh -l nodes=1:tank-870:i5-6500te -F "$VIDEO ../results/ FP16 2 1"
print(job_id_gpu[0])

#Progress indicators
if job_id_gpu:
    progressIndicator('../results/', 'i_progress_'+job_id_gpu[0]+'.txt', "Inference", 0, 100)

### Submitting to an edge compute node with Intel® VPU

In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank 870-Q170</a> edge node with an <a href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel Core™ i5-6500te CPU</a> . The inference workload will run on the <a href="https://www.ieiworld.com/mustang-v100/en/"> IEI Mustang-V100-MX8 </a> card installed in this node.

In [ ]:
print("Submitting job to node with Intel VPU...")
#Submit job to the queue
job_id_hddl_fp16 = !qsub shopper_gaze.sh -l nodes=1:tank-870:iei-mustang-v100-mx8 -F "$VIDEO ../results/ FP16 2 3"
print(job_id_hddl_fp16[0]) 
#Progress indicators
if job_id_hddl_fp16:
    progressIndicator('../results/', 'i_progress_'+job_id_hddl_fp16[0]+'.txt', "Inference", 0, 100)

### Submitting to an edge compute node with IEI Mustang-F100-A10 (Intel® Arria® 10 FPGA)¶

In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank 870-Q170</a> edge node with an <a href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel Core i5-6500te CPU</a> . The inference workload will run on the <a href="https://www.ieiworld.com/mustang-f100/en/"> IEI Mustang-F100-A10 </a> card installed in this node.

In [ ]:
print("Submitting job to node with Intel FPGA HDDL-F...")
#Submit job to the queue
job_id_fpga = !qsub shopper_gaze.sh -l nodes=1:tank-870:i5-6500te:iei-mustang-f100-a10 -F "$VIDEO ../results/ FP32 2 5"
print(job_id_fpga[0])

#Progress indicators
if job_id_fpga:
    progressIndicator('../results/', 'i_progress_'+job_id_fpga[0]+'.txt', "Inference", 0, 100)

### Submitting to an edge compute node with Intel® Movidius™ Neural Compute Stick 2
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank 870-Q170</a> edge node with an <a href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel Core i5-6500te CPU</a>. The inference workload will run on an <a 
    href="https://software.intel.com/en-us/neural-compute-stick">Intel Neural Compute Stick 2</a> installed in this  node.

In [ ]:
print("Submitting job to node with Intel NCS2...")
#Submit job to the queue
job_id_ncs2 = !qsub shopper_gaze.sh -l nodes=1:tank-870:i5-6500te:intel-ncs2 -F "$VIDEO ../results/ FP16 2 3"
print(job_id_ncs2[0])

#Progress indicators
if job_id_ncs2:
    progressIndicator('../results/', 'i_progress_'+job_id_ncs2[0]+'.txt', "Inference", 0, 100)

### Submitting to an edge compute node with UP Squared Grove IoT Development Kit (UP2)
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/up-squared-grove-dev-kit">UP Squared Grove IoT Development Kit</a> edge node with an <a 
    href="https://ark.intel.com/products/96488/Intel-Atom-x7-E3950-Processor-2M-Cache-up-to-2-00-GHz-">Intel Atom® x7-E3950 Processor</a>. The inference  workload will run on the integrated Intel® HD Graphics 505 card.

In [ ]:
#Submit job to the queue
job_id_up2 = !qsub shopper_gaze.sh -l nodes=1:up-squared -F "$VIDEO ../results/ FP32 2 1"
print(job_id_up2[0]) 
#Progress indicators
if job_id_up2:
    progressIndicator('../results/', 'i_progress_'+job_id_up2[0]+'.txt', "Inference", 0, 100)

## Check if the jobs are done

To check on the jobs that were submitted, use the `qstat` command.

We have created a custom Jupyter widget  to get live qstat update.
Run the following cell to bring it up.

In [ ]:
liveQstat()

You should see the jobs that you have submitted (referenced by `Job ID` that gets displayed right after you submit the job in step 2.3).
There should also be an extra job in the queue "jupyterhub": this job runs your current Jupyter Notebook session.

The 'S' column shows the current status. 
- If it is in Q state, it is in the queue waiting for available resources. 
- If it is in R state, it is running. 
- If the job is no longer listed, it means it is completed.

**Note**: Time spent in the queue depends on the number of users accessing the edge nodes. Once these jobs begin to run, they should take from 1 to 5 minutes to complete. 

***Wait!***

Please wait for the inference jobs and video rendering complete before proceeding to the next step.

## Step 3: View Results

Once the jobs are completed, the queue system outputs the stdout and stderr streams of each job into files with names of the form

`obj_det_{type}.o{JobID}`

`obj_det_{type}.e{JobID}`

(here, obj_det_{type} corresponds to the `-N` option of qsub).

However, for this case, we may be more interested in the output video files. They are stored in mp4 format inside the `results/` directory.
We wrote a short utility script that will display these videos with in the notebook.
Run the cells below to display them.
See `demoutils.py` if you are interested in understanding further how the results are displayed in notebook.

In [ ]:
videoHTML('IEI Tank Core (Intel Core CPU)',
          ['results/output_'+job_id_core[0]+'.mp4'],
          '../results/stats_'+job_id_core[0]+'.txt')

In [ ]:
videoHTML('IEI Tank Core (Intel Xeon CPU)',
          ['results/output_'+job_id_xeon[0]+'.mp4'],
          '../results/stats_'+job_id_xeon[0]+'.txt')

In [ ]:
videoHTML('IEI Intel GPU (Intel Core + Onboard GPU)', 
          ['results/output_'+job_id_gpu[0]+'.mp4'],
          '../results/stats_'+job_id_gpu[0]+'.txt')

In [ ]:
videoHTML('IEI Tank Core (Intel VPU - Mustang v100-mx8)',
          ['results/output_'+job_id_hddl_fp16[0]+'.mp4'],
          '../results/stats_'+job_id_hddl_fp16[0]+'.txt')

In [ ]:
videoHTML('Tank-870:i5-6500te (Intel® NCS2)',
          ['results/output_'+job_id_ncs2[0]+'.mp4'],
          '../results/stats_'+job_id_ncs2[0]+'.txt')

In [ ]:
videoHTML('IEI Tank + IEI Mustang-F100-A10 (Intel® Arria® 10 FPGA)',
          ['results/output_'+job_id_fpga[0]+'.mp4'],
          '../results/stats_'+job_id_fpga[0]+'.txt')

In [ ]:
videoHTML('UP Squared Grove IoT Development Kit (UP2)',
          ['results/output_'+job_id_up2[0]+'.mp4'],
          '../results/stats_'+job_id_up2[0]+'.txt')

## Step 4: Assess Performance

The running time of each inference task is recorded in `results/*/stats.txt`, where the subdirectory name corresponds to the architecture of the target edge compute node. Run the cell below to plot the results of all jobs side-by-side. Lower values mean better performance. Keep in mind that some architectures are optimized for the highest performance, others for low power or other metrics.

In [ ]:
arch_list = [('core', 'Intel Core\ni5-6500TE\nCPU'),
             ('xeon', 'Intel Xeon\ne3-1268l-v5'),
             ('gpu', ' Intel Core\ni5-6500TE\nGPU'),
              ('fpga', 'Intel FPGA\nMustang-\nF100-A10'),
             ('ncs2', 'Intel\nNCS2'),
            ('hddl_fp16', 'Intel VPU\nMustang-\nv100-mx8'),   
            ('up2', 'Intel Atom\nUP2/GPU')]

stats_list = []
for arch, a_name in arch_list:
    if 'job_id_'+arch in vars():
        #stats_list.append(('../results/{arch}/stats.txt'.format(arch=arch), a_name))
        stats_list.append(('../results/stats_'+vars()['job_id_'+arch][0]+'.txt', a_name))
    else:
        stats_list.append(('placeholder'+arch, a_name))

summaryPlot(stats_list, 'Architecture', 'Time, seconds', 'Inference Engine Processing Time', 'time' )
summaryPlot(stats_list, 'Architecture', 'Frames per second', 'Inference Engine FPS', 'fps' )